### Stacking para Clasificación

In [ ]:
# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

Usamos la función make_classification() para crear un problema de clasificación binaria sintética con 1,000 ejemplos y 20 características de entrada.

In [ ]:
# get the dataset
def get_dataset():
    X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
    return X, y
    print(X.shape, y.shape)

La función get_stacking() a continuación define el modelo StackingClassifier definiendo primero una lista de tuplas para los cinco modelos base, luego definiendo el metamodelo de regresión logística para combinar las predicciones de los modelos base usando una validación cruzada de 5 veces.

In [ ]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression()))
    level0.append(('knn', KNeighborsClassifier()))
    level0.append(('cart', DecisionTreeClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
    # define meta learner model
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

Incluimos el conjunto de apilamiento en la lista de modelos para evaluar, junto con los modelos independientes.

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier()
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    models['stacking'] = get_stacking()
    return models

Nuestra expectativa es que el conjunto de apilamiento funcione mejor que cualquier modelo de base única.
Este no es siempre el caso y si no es el caso, entonces el modelo base debe usarse a favor del modelo de conjunto.

In [ ]:
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [ ]:
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
    #plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

En este caso, podemos ver que el conjunto de stacking parece funcionar mejor que cualquier modelo individual en promedio, logrando una precisión de aproximadamente el **96,4** por ciento.

### Stacking para Regresión

In [ ]:
# compare machine learning models for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot

In [ ]:
# get the dataset
def get_dataset():
	X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=1)
	return X, y

Cada algoritmo se evaluará utilizando los hiperparámetros del modelo predeterminados. La función get_models() a continuación crea los modelos que deseamos evaluar.

In [ ]:
# get a list of models to evaluate
def get_models():
	models = dict()
	models['knn'] = KNeighborsRegressor()
	models['cart'] = DecisionTreeRegressor()
	models['svm'] = SVR()
	return models

Cada modelo se evaluará mediante la validación cruzada repetida de k veces. La función evaluate_model() a continuación toma una instancia de modelo y devuelve una lista de puntuaciones de tres repeticiones de validación cruzada de 10 veces.

In [ ]:
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
	return scores

En este caso, el rendimiento del modelo se informará utilizando el error absoluto medio (MAE). La biblioteca scikit-learn invierte el signo de este error para maximizarlo, de -infinito a 0 para obtener la mejor puntuación.

In [ ]:
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

Ejecutar el ejemplo primero informa la media y la desviación estándar MAE (mean absolute error) para cada modelo. Podemos ver que en este caso, KNN se desempeña mejor con un MAE negativo medio de alrededor de ***-100.***